In [1]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC


In [2]:
data = pd.read_csv('ts.csv')

In [3]:
data['text_length']=data['tweet'].apply(len)

In [4]:
tweet=data.tweet

# Preprocessing

In [5]:
import re
from nltk.stem.porter import PorterStemmer
stopwords = nltk.corpus.stopwords.words("english")
stemmer = PorterStemmer()
#ff(follow friday)#rt(retweet)Mt(modified tweet)
others = ["#ff", "rt","MT"]
#extending stopwords
stopwords.extend(others)
def preprocess(tweet):  
    
    # removing extra spaces
    ex= re.compile(r'\s+')
    tweet_space = tweet.str.replace(ex, ' ')
    
    # removing '@'(mentions)
    ex = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(ex, '')
    
    # removing url's (https://xyz.com)
    gex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|' '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(gex, '')
    
    # removing punctuations and numbers
    punctuation_remove = tweets.str.replace("[^a-zA-Z]", " ")
    
    # removing whitespace 
    newtweet=punctuation_remove.str.replace(r'\s+', ' ')
    
    # removing leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    
    # lowercasing the tweet
    tl = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tl.apply(lambda x: x.split())
    
    # removing stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [i for i in x if i not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tp= tokenized_tweet
    
    return tp

processed_tweets = preprocess(tweet)   

data['processed_tweets'] = processed_tweets

C:\Users\Kowshik\AppData\Local\Temp\ipykernel_21988\4027588305.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  punctuation_remove = tweets.str.replace("[^a-zA-Z]", " ")
C:\Users\Kowshik\AppData\Local\Temp\ipykernel_21988\4027588305.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=punctuation_remove.str.replace(r'\s+', ' ')
C:\Users\Kowshik\AppData\Local\Temp\ipykernel_21988\4027588305.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
C:\Users\Kowshik\AppData\Local\Temp\ipykernel_21988\4027588305.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')


In [6]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.80, min_df=5, max_features=1000)
tfidf = tfidf_vectorizer.fit_transform(data['processed_tweets'] )


In [7]:
X = tfidf
y = data['label'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.45)

# Logistic Regression

In [8]:
model = LogisticRegression().fit(X_train,y_train)
y_preds = model.predict(X_test)
acc=accuracy_score(y_test,y_preds)
print("Logistic Regression")
print("Accuracy score: %f"%(acc*100)+"%")

Logistic Regression
Accuracy score: 94.521310%


# Naive Bayes

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, random_state=42, test_size=0.2)

nb=GaussianNB()
nb.fit(X_train,y_train)
y_preds = nb.predict(X_test)
acc2=accuracy_score(y_test,y_preds)
print("Naive Bayes")
print("Accuracy Score:%f"%(acc2*100)+"%")

Naive Bayes
Accuracy Score:51.853590%


# Random Forest 

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train, y_train)
y_preds = rf.predict(X_test)
acc1=accuracy_score(y_test,y_preds)
print("Random Forest")
print("Accuracy score: %f"%(acc1*100)+"%")

In [ ]:
support =LinearSVC(random_state=20)
support.fit(X_train,y_train)
y_preds = support.predict(X_test)
acc3=accuracy_score(y_test,y_preds)
print("SVM")
print("Accuracy Score:%f"%(acc3*100)+"%")

In [ ]:
import matplotlib.pyplot as plt
objects = ('Logistic', 'RandomForest', 'Naive_bayes', 'SVM')
y_pos = np.arange(len(objects))
performance = [acc,acc1,acc2,acc3]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.title('Algorithm Comparision for F1')
plt.show()

In [ ]:
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,max_features=1000,stop_words= ENGLISH_STOP_WORDS)),
                            ('model',RandomForestClassifier())])

In [ ]:
pipeline.fit(data.processed_tweets,data.label)

In [ ]:
pipeline.predict(data.processed_tweets)

In [22]:
from joblib import dump

In [23]:
dump(pipeline, filename="text_classification.joblib")

['text_classification.joblib']

In [24]:
data[data.label == 1]

,id,label,tweet,text_length,processed_tweets
13,14,1,@user #cnn calls #michigan middle school 'buil...,74,cnn call michigan middl school build wall chan...
14,15,1,no comment! in #australia #opkillingbay #se...,101,comment australia opkillingbay seashepherd hel...
17,18,1,retweet if you agree!,22,retweet agre
23,24,1,@user @user lumpy says i am a . prove it lumpy.,47,lumpi say prove lumpi
34,35,1,it's unbelievable that in the 21st century we'...,104,unbeliev st centuri need someth like neverump ...
...,...,...,...,...,...
31934,31935,1,lady banned from kentucky mall. @user #jcpenn...,59,ladi ban kentucki mall jcpenni kentucki
31946,31947,1,@user omfg i'm offended! i'm a mailbox and i'...,82,omfg offend mailbox proud mailboxprid liberalism
31947,31948,1,@user @user you don't have the balls to hashta...,112,ball hashtag say weasel away lumpi toni dipshit
31948,31949,1,"makes you ask yourself, who am i? then am i a...",87,make ask anybodi god oh thank god
